In [3]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 
from tqdm import tqdm_notebook


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

from ngboost.ngboost import NGBoost
from ngboost.learners import default_tree_learner
from ngboost.distns import Normal
from ngboost.scores import MLE

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:115: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [0]:
from pydrive.auth import GoogleAuth

from pydrive.drive import GoogleDrive

from google.colab import auth

from oauth2client.client import GoogleCredentials 


# 1. Authenticate and create the PyDrive client.

auth.authenticate_user()

gauth = GoogleAuth()

gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

In [0]:
file_id = '1XATewioAnfsVMcZiGN6AYh9ZsMRFX4Yd'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('TEST_DHweightDaconArima_NAFilled.csv')

In [0]:
file_id = '1Kkv55I6cT7DgEFyeC-ryoq2wi3Aq2JSI'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('hourly.csv')

In [7]:
# hourly WEATHER data
w_hr = pd.read_csv("hourly.csv",encoding='euc-kr', sep=',')
w_hr2 = w_hr.loc[:,['Time', 'Temp','Humidity', 'WindSpeed']]
w_hr2['Humidity']=w_hr2.loc[:,'Humidity'].fillna(0)
w_hr2.head(2)

Time  Temp  Humidity  WindSpeed
0  2016.7.20 0:00  25.7       0.0        1.3
1  2016.7.20 1:00  25.6       0.0        1.4

In [8]:
w_hr2['Time'] = pd.to_datetime(w_hr2.Time)
w_hr2 = w_hr2.set_index('Time')
w_hr2.head(2)

Temp  Humidity  WindSpeed
Time                                          
2016-07-20 00:00:00  25.7       0.0        1.3
2016-07-20 01:00:00  25.6       0.0        1.4

In [0]:
test = pd.read_csv("TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [0]:
test.index = pd.to_datetime(test.index)

In [0]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [0]:
TE_time_vars = dateTimeVar(test)

In [0]:
submission_hourly = pd.date_range(start = pd.datetime(2018, 7, 1, 0),
                             end = pd.datetime(2018, 7, 1, 23),
                             freq = "H")
tmp_test = pd.DataFrame(index = submission_hourly)

In [0]:
X_test_H = dateTimeVar(tmp_test)

In [0]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [0]:
Mer = pd.concat([TE_time_vars, X_test_H])
Mer1 = dummies(Mer, ['season', 'dayClass'])
TE_time_vars1 = Mer1.iloc[:7334].copy()
X_test_H1 = Mer1.iloc[7334:7358].copy()

In [0]:
X_test_H = pd.concat([X_test_H1, w_hr2], axis=1,join='inner')

In [0]:
TE_timeV_dum = TE_time_vars1.copy()

In [0]:
TE_timeV_dum_wthr= pd.concat([TE_timeV_dum, w_hr2], axis=1,join='inner')

In [0]:
from sklearn.model_selection import GridSearchCV


lg_param = {"eta" : [0.2],
           "Min_child_weight" : [1],
           "Max_depth" : [3]}

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    model.fit(np.asarray(X_train), np.asarray(y_train))
    fcst = model.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[i]]
    return a
   
def test50_Hrly_GS_LGBReg(train, X_test_H):
    hrly_50 = pd.DataFrame()
    for mID in tqdm_notebook(train.columns):
        data = pd.concat([train.loc[:,[mID]], TE_timeV_dum_wthr], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test_H['lag_24'] = y_train.tail(24).values
        X_test_H['lag_168'] = data.loc[:,'lag_168'][7166:7190].values
        
        model = NGBoost(Base=default_tree_learner, Dist=Normal, Score=MLE(), natural_gradient=True,verbose=False)
        tmp = gsLGBR_Hrly_FitPred(model, lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly_50 = pd.concat([hrly_50, tmp], axis=0)
    return hrly_50

In [25]:
pred_2 = test50_Hrly_GS_LGBReg(test, X_test_H)

In [0]:
pred_2.to_csv("ngb1.csv", index=False)

In [33]:
!ls -lh 

total 16M
-rw-r--r-- 1 root root 2.6K Oct 31 08:11 adc.json
-rw-r--r-- 1 root root 692K Oct 31 08:13 hourly.csv
-rw-r--r-- 1 root root  93K Oct 31 08:57 ngb1.csv
-rw-r--r-- 1 root root  93K Oct 31 08:55 ngb.csv
drwxr-xr-x 1 root root 4.0K Oct 25 16:58 sample_data
-rw-r--r-- 1 root root  16M Oct 31 08:11 TEST_DHweightDaconArima_NAFilled.csv


In [0]:
from google.colab import files
files.download('ngb1.csv')